In [2]:
import random
import sqlite3
import datetime
import numpy as np
import pandas as pd
pd.set_option("display.max_rows", None)
import os
import glob
import cv2
import imutils
from PIL import Image
import easyocr
import time
from matplotlib import pyplot as plt
from google.colab.patches import cv2_imshow
#from google.colab import drive
#drive.mount('/content/drive')
conn = sqlite3.connect('parking.db')
import warnings
warnings.filterwarnings('ignore')

In [1]:
#!pip install easyocr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.9 MB 5.4 MB/s 
     |████████████████████████████████| 604 kB 66.5 MB/s 
     |████████████████████████████████| 108 kB 67.8 MB/s 
     |████████████████████████████████| 47.6 MB 1.9 MB/s 
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.6.0.66
    Uninstalling opencv-python-headless-4.6.0.66:
      Successfully uninstalled opencv-python-headless-4.6.0.66


Creation of databases

In [3]:
#c = conn.cursor()
def create_database(c):
  c.execute("DROP TABLE IF EXISTS VEHICLE")
  vehicle =""" CREATE TABLE VEHICLE(
    Vehicle_id INTEGER PRIMARY KEY,
    Vehicle_number  VARCHAR(25) NOT NULL,
    Vehicle_type VARCHAR(10) NOT NULL
    ); """
  c.execute(vehicle)
  c.execute("DROP TABLE IF EXISTS VEHICLELOG")
  log = """ CREATE TABLE VEHICLELOG(
    Log_id INTEGER PRIMARY KEY,
    Vehicle_id INTEGER NOT NULL,
    Rate_id INTEGER NOT NULL,
    Entry TEXT NOT NULL,
    Exit  TEXT,
    Bill  REAL 
    ); """
  c.execute(log)
  c.execute("DROP TABLE IF EXISTS VEHICLETYPERATE")
  vehicletype =""" CREATE TABLE VEHICLETYPERATE(
    Rate_id INTEGER PRIMARY KEY,
    Vehicle_type CHAR(10) NOT NULL,
    Rate  REAL NOT NULL
    ); """
  c.execute(vehicletype)
  c.execute("INSERT INTO VEHICLETYPERATE(Vehicle_type,Rate) VALUES(?,?)",('car',30))
  c.execute("INSERT INTO VEHICLETYPERATE(Vehicle_type,Rate) VALUES(?,?)",('motorbike',20))
  conn.commit()

In [4]:
def insert_database(c,type,number,run_number):
  number = number.strip()
  str1 = "SELECT COUNT(1) FROM VEHICLE WHERE Vehicle_number ='" + number +"'"
  c.execute(str1)
  rows = c.fetchall()
  for row in rows:
    temp = row[0]
  if temp==0:
     c.execute("INSERT INTO VEHICLE(Vehicle_number,Vehicle_type) VALUES(?, ?)",(number,type))
  else:
    number1 = number + str(run_number)
    number = str(number1)
    c.execute("INSERT INTO VEHICLE(Vehicle_number,Vehicle_type) VALUES(?, ?)",(number,type))
  str1 = "SELECT vehicle_id FROM VEHICLE WHERE Vehicle_number ='" + number +"'"
  c.execute(str1)
  rows = c.fetchall()
  for row in rows:
    temp = row[0]
  vehicleid = str(temp)
  str1="SELECT COUNT(1) FROM VEHICLELOG WHERE Vehicle_id ='" + vehicleid + "' and exit is null"
  c.execute(str1)
  rows = c.fetchall()
  for row in rows:
    temp = row[0]
  if temp>0:
      print("Vehicle " + number+ " has already entered. Cannot enter again")
  else:
    str1 ="select a.vehicle_id,b.rate_id from vehicle a,vehicletyperate b where a.vehicle_number = '"+number+"' and a.vehicle_type=b.vehicle_type"
    c.execute(str1)
    rows = c.fetchall()
    for row in rows:
      vehicleid = row[0]
      rateid = row[1]
    c.execute("INSERT INTO VEHICLELOG(Vehicle_id,Rate_id,Entry) VALUES(?, ?,?)",(vehicleid,rateid,datetime.datetime.now()))
    conn.commit()

In [5]:
def update_database(c,number):
  close_gate()
  number = number.strip()
  str1 = "SELECT vehicle_id FROM VEHICLE WHERE Vehicle_number ='" + number +"'"
  c.execute(str1)
  rows = c.fetchall()
  for row in rows:
    temp = row[0]
  temp = str(temp)
  str1="SELECT COUNT(1) FROM VEHICLELOG WHERE Vehicle_id ='" + temp + "' and exit is null"
  c.execute(str1)
  rows = c.fetchall()
  for row in rows:
    temp = row[0]
  if temp==0:
    print("Vehicle " +number+ " has already exited")
  else:
    str1 = "select a.rate,b.entry,b.log_id from vehicletyperate a, vehiclelog b, vehicle c where c.vehicle_number ='" +number+"'  and b.vehicle_id = c.vehicle_id and b.rate_id = a.rate_id and b.exit is null"
    c.execute(str1)
    exit = datetime.datetime.now()
    rows = c.fetchall()
    for row in rows:
      rate = row[0]
      entry = row[1]
      logid = row[2]
    entrytime=datetime.datetime.strptime(entry, '%Y-%m-%d %H:%M:%S.%f')
    time=exit-entrytime
    bill_amt = (int(time.total_seconds()/3600)+1)*rate
    exittime = datetime.datetime.strftime(exit, '%Y-%m-%d %H:%M:%S.%f')
    bill_amt = str(bill_amt)
    logid = str(logid)
    print_bill(bill_amt,number)
    str1 = "update vehiclelog set exit ='" +exittime+"', bill = '" +bill_amt+"' where log_id ='" +logid+"'"
    open_gate()
    c.execute(str1) 
    conn.commit()
  print(" ")           

In [6]:
def open_gate():
  print("Gate opened")

In [7]:
def close_gate():
  print("Gate closed")

In [8]:
def print_bill(amt,number):
  print(number)
  print("Pay Rs"+amt)

In [9]:
def view_database(c):
  pd.set_option('display.width', 1000)
  cur1 = """SELECT * FROM VEHICLE;"""
  c.execute(cur1)
  df1 = pd.read_sql_query("SELECT * FROM VEHICLE",conn)
  print(df1)
  print("----------------------------------------------------------------------------------------------------")
  cur2 = """SELECT * FROM VEHICLETYPERATE;"""
  c.execute(cur2)
  df2 = pd.read_sql_query("SELECT * FROM VEHICLETYPERATE",conn)
  print(df2)
  print("----------------------------------------------------------------------------------------------------")
  cur3 = """SELECT * FROM VEHICLELOG;"""
  c.execute(cur3)
  df3 = pd.read_sql_query("SELECT * FROM VEHICLELOG",conn)
  print(df3)

Vehicle Type Detection

In [10]:
config_file = '/content/drive/MyDrive/ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt'
frozen_model = '/content/drive/MyDrive/frozen_inference_graph.pb'
model = cv2.dnn_DetectionModel(frozen_model,config_file)
classLabel = []
file_name = '/content/drive/MyDrive/Labels.txt'
with open(file_name,'rt') as fpt:
  classLabel = fpt.read().rstrip('\n').split('\n')
#print(classLabel)
model.setInputSize(320,320)
model.setInputScale(1.0/127.5)
model.setInputMean((127.5,127.5,127.5))
model.setInputSwapRB(True)
def vehicle_type_detection(fname):
  str = '/content/drive/MyDrive/images/' + fname
  #str = '/content/Cars0.png'
  image1 = cv2.imread(str.rstrip())
  #cv2_imshow(image1)
  gray = cv2.cvtColor(image1,cv2.COLOR_BGR2GRAY)
  ClassIndex, confidence, bbox = model.detect(image1,confThreshold=0.5)
  str1="car"
  if classLabel[ClassIndex[0]-1]!='car' and classLabel[ClassIndex[0]-1]!='motorbike':
    return str1
  return classLabel[ClassIndex[0]-1]

Vehicle plate detection

In [11]:
def plate_detection(fname):
  str = '/content/drive/MyDrive/images/' + fname
  #str = '/content/Cars0.png'
  image1 = cv2.imread(str.rstrip())
  gray = cv2.cvtColor(image1,cv2.COLOR_BGR2GRAY)
  blur = cv2.bilateralFilter(gray,11,17,17)
  edges = cv2.Canny(blur,30,200)
  key = cv2.findContours(edges.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
  cnts = imutils.grab_contours(key)
  cnts = sorted(cnts, key=cv2.contourArea, reverse=True)[:10]
  plate = None
  for c in cnts:
    edges_count = cv2.approxPolyDP(c,10, True)
    if len(edges_count) == 4:
      plate=edges_count
      break
  mask = np.zeros(gray.shape, np.uint8)
  new=cv2.drawContours(mask, [plate],0,255,-1)
  new=cv2.bitwise_and(image1,image1,mask=mask)
  (x,y) = np.where(mask==255)
  (x1,y1) = (np.min(x), np.min(y))
  (x2,y2) = (np.max(x), np.max(y))
  img = gray[x1:x2+1 , y1:y2+1]
  res = cv2.rectangle(image1, tuple(edges_count[0][0]),tuple(edges_count[2][0]),(0,255,0),3)
  #cv2_imshow(res)
  #plt.figure()
  cv2.imwrite("plate.png",img)
  reader = easyocr.Reader(['en'], gpu=False, verbose=False)
  text = reader.readtext('/content/plate.png')
  str=""
  x=len(text)
  for i in range(0,x):
    str=str+text[i][-2]
  return str

Reading images from text files

In [12]:
def read_image_entry(c,file_path):
  str = "/content/drive/MyDrive/" + file_path
  f = open(str,'r')
  line = f.readlines()
  run_number=1
  for i in line:
    type = vehicle_type_detection(i)
    number = plate_detection(i)
    insert_database(c,type,number,run_number)
    run_number=run_number+1
  f.close()

In [13]:
def read_image_exit(c,file_path):
  str = "/content/drive/MyDrive/" + file_path
  f = open(str,'r')
  line = f.readlines()
  for i in line:
    number = plate_detection(i)
    update_database(c,number)
  
  f.close()

In [14]:
c = conn.cursor()
create_database(c)
st = time.time()
read_image_entry(c,"Entry.txt")
et = time.time()
print('Execution time: ',(et-st))
view_database(c)

Execution time:  348.03207087516785
    Vehicle_id     Vehicle_number Vehicle_type
0            1        KL 65 H4383          car
1            2         KL01CA2555          car
2            3           DZIZ YXR          car
3            4        MH32RaF4884    motorbike
4            5            BCF 260          car
5            6    AB MH-08At 3000    motorbike
6            7        MH13 BN8454          car
7            8      AP39S8889LrV=    motorbike
8            9           GRI3 GGS          car
9           10        HHI2 DE1433          car
10          11         HH13BK8100          car
11          12         oD02AT8301          car
12          13      NH 13 CD 0096          car
13          14        @LZC N 5617          car
14          15       MH 20EJ 0364          car
15          16       MH 20EE 7598          car
16          17           Czi7 kod          car
17          18            DAN 54P          car
18          19           PGoMN112          car
19          20        P6

In [15]:
#c = conn.cursor()
read_image_exit(c,"Exit.txt")
view_database(c)
c.close()

Gate closed
KL 65 H4383
Pay Rs60.0
Gate opened
 
Gate closed
KL01CA2555
Pay Rs60.0
Gate opened
 
Gate closed
DZIZ YXR
Pay Rs60.0
Gate opened
 
Gate closed
MH32RaF4884
Pay Rs40.0
Gate opened
 
Gate closed
BCF 260
Pay Rs60.0
Gate opened
 
Gate closed
AB MH-08At 3000
Pay Rs40.0
Gate opened
 
Gate closed
MH13 BN8454
Pay Rs60.0
Gate opened
 
Gate closed
AP39S8889LrV=
Pay Rs40.0
Gate opened
 
Gate closed
GRI3 GGS
Pay Rs60.0
Gate opened
 
Gate closed
HHI2 DE1433
Pay Rs60.0
Gate opened
 
Gate closed
HH13BK8100
Pay Rs60.0
Gate opened
 
Gate closed
oD02AT8301
Pay Rs60.0
Gate opened
 
Gate closed
NH 13 CD 0096
Pay Rs60.0
Gate opened
 
Gate closed
@LZC N 5617
Pay Rs60.0
Gate opened
 
Gate closed
MH 20EJ 0364
Pay Rs60.0
Gate opened
 
Gate closed
MH 20EE 7598
Pay Rs60.0
Gate opened
 
Gate closed
Czi7 kod
Pay Rs60.0
Gate opened
 
Gate closed
DAN 54P
Pay Rs60.0
Gate opened
 
Gate closed
PGoMN112
Pay Rs60.0
Gate opened
 
Gate closed
P6J03JLO126
Pay Rs60.0
Gate opened
 
    Vehicle_id     Vehicle_number